In [266]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools
pd.options.display.max_rows = 5000
pd.set_option('display.max_colwidth', 255)


The data source for this analysis is a [Kaggle notebook](https://www.kaggle.com/heesoo37/120-years-of-olympic-history-athletes-and-results), which in turns was made with data scraped from the former Olympics Reference website, now [Olympmedia](http://www.olympedia.org/).

In [241]:
# Reads the data in
df = pd.read_csv("../data/athlete_events.csv")

We only want to look at the Summer Olympics, so let's filter the dataset.

In [242]:
# Filters the dataset, keeping only Summer games
df = df[df.Season == "Summer"].reset_index(drop=True)

In [243]:
# Removes the non-recognized 1906 Games
df = df[df.Games!="1906 Summer"].reset_index(drop=True)

In [244]:
# Removes athletes that participated in art or demonstrastive competitions
df = df[~df.Sport.isin([
    "Art Competitions", "Aeronautics", "Alpinism",
])].reset_index(drop=True)

#### Women participation over time in 

First, let's take a look at the ammoun of women athletes in each Olympics. To do so, we must remove duplicate entries for athletes (since many of them participated in many events or in many Olympics).

In [245]:
# Keep one entry for each athlete in each Games he/she competed in
atl_games = df.drop_duplicates(subset=['ID', "Games"])

In [246]:
# Groups and counts
gp_atl_games = (atl_games.groupby(["Games", "Sex", "Year"])
                    .count()
                    .ID
                    .reset_index()
                    .rename(columns={"ID": "No_athletes"}))

In [247]:
gp_atl_games.head(100)

,Games,Sex,Year,No_athletes
0,1896 Summer,M,1896,176
1,1900 Summer,F,1900,23
2,1900 Summer,M,1900,1201
3,1904 Summer,F,1904,6
4,1904 Summer,M,1904,644
5,1908 Summer,F,1908,44
6,1908 Summer,M,1908,1980
7,1912 Summer,F,1912,53
8,1912 Summer,M,1912,2324
9,1920 Summer,F,1920,77


Let's see that tendency in a split bar chart.

In [248]:
fig = go.Figure()
fig.add_trace(go.Bar(x=gp_atl_games[gp_atl_games.Sex=="M"].Year,
                y=gp_atl_games[gp_atl_games.Sex=="M"].No_athletes,
                name='Men',
                marker_color='rgb(55, 83, 109)'
                ))

fig.add_trace(go.Bar(x=gp_atl_games[gp_atl_games.Sex=="F"].Year,
                y=gp_atl_games[gp_atl_games.Sex=="F"].No_athletes,
                name='Women',
                marker_color='rgb(26, 118, 255)'
                ))

fig.update_layout(
    title='Athletes by gender in each Summer Olympics',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=.5, # gap between bars of adjacent location coordinates.
    bargroupgap=0 # gap between bars of the same location coordinate.
)
fig.show()


Alternatively, we can better visualize this pattern using percentages and a single line.

In [249]:
# Pivot table
gp_games_percentage = pd.pivot_table(gp_atl_games, values='No_athletes', index=['Games', 'Year'],
                                     columns=['Sex'], aggfunc=np.sum).reset_index()

In [250]:
# Crates a field for women percentage
gp_games_percentage["Percentage_women"] = (gp_games_percentage["F"] / 
                                          (gp_games_percentage["F"] + gp_games_percentage["M"]))

In [251]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=gp_games_percentage.Year,
                y=gp_games_percentage.Percentage_women,
                mode='lines+markers',
                marker_color='rgb(55, 83, 109)',
                showlegend=False))

# Adds a 50% line
fig.add_trace(go.Scatter(x=[1896, 2020], 
                         y=[.5, .5],
                         mode='lines',
                         showlegend=False,
                         marker_color="#222"))

fig.update_xaxes(range=[1896, 2020])
fig.update_yaxes(range=[0, .55])


We can see that the ammount of female athletes is increasing steadily since the 1980 games, with a minor hiccup in the London 2012.  

Let's take a loot at this evolution for some Olympic powerhouses: USA, USSR, Great Britain, France, Germany, Italy, China, Australia, Sweden, Hungary, Japan, East Germany, Russia, Romenia, Finland, Canada, Netherlands, Poland, South Korea and Cuba. They are the top 20 in historic Olympic medal tables. Note that we are not aggregating "heir" nations (Germany data will not include athletes from West Germany, for instance).

We will fill the list with the nations that are not among the historical top 20, but that are among the top 30 in the last Summer Games.


In [252]:
# Creates a more detailed pivot table
noc_games = (atl_games.groupby(["Games", "Sex", "Year", "NOC"])
                    .count()
                    .ID
                    .reset_index()
                    .rename(columns={"ID": "No_athletes"}))

In [253]:
# Makes a pivot table
noc_games_sex = pd.pivot_table(noc_games, values='No_athletes', index=['Games', 'NOC', 'Year'],
                    columns=['Sex'], aggfunc=np.sum).reset_index()

In [254]:
# Crates a field for women percentage
noc_games_sex["Percentage_women"] = noc_games_sex["F"] / (noc_games_sex["F"] + noc_games_sex["M"])

In [255]:
# Fills NaN with zeroes
noc_games_sex = noc_games_sex.fillna(0)
noc_games_sex.head()

Sex,Games,NOC,Year,F,M,Percentage_women
0,1896 Summer,AUS,1896,0.0,1.0,0.0
1,1896 Summer,AUT,1896,0.0,3.0,0.0
2,1896 Summer,DEN,1896,0.0,3.0,0.0
3,1896 Summer,FRA,1896,0.0,12.0,0.0
4,1896 Summer,GBR,1896,0.0,10.0,0.0


In [256]:
# Plots many small multiples
countries = [
             'USA', 'URS', 'GBR', 'FRA', 'GER',
             'ITA', 'CHN', 'AUS', 'SWE', 'HUN',
             'JPN', 'RUS', 'GDR', 'ROU', 'FIN',
             'CAN', 'NED', 'POL', 'KOR', 'CUB',
             'ESP', 'KEN', 'JAM', 'CRO', 'NZL', 
             'UZB', 'KAZ', 'IRI', 'COL', 'SUI', 
             'GRE', 'ARG', 'DEN', 'RSA', 'BRA',
            ]

positions = [
             (1,1), (1,2), (1,3), (1,4), (1,5),
             (2,1), (2,2), (2,3), (2,4), (2,5),
             (3,1), (3,2), (3,3), (3,4), (3,5),
             (4,1), (4,2), (4,3), (4,4), (4,5),
             (5,1), (5,2), (5,3), (5,4), (5,5),
             (6,1), (6,2), (6,3), (6,4), (6,5),
             (7,1), (7,2), (7,3), (7,4), (7,5),
    ]


fig = make_subplots(rows=7, cols=5, subplot_titles=countries)


assert len(countries) == len(positions)

for NOC, position in zip(countries, positions):
    
    subset = noc_games_sex[noc_games_sex.NOC==NOC]
    subset = subset.sort_values(by='Year')
    
    row = position[0]
    col = position[1]

    fig.add_trace(go.Scatter(x=subset.Year,
                    y=subset.Percentage_women,
                    mode='lines',
                    marker_color='rgb(55, 83, 109)',
                    showlegend=False),
                    row=row,
                    col=col,
                )
    
    # Adds a 50% line
    fig.add_trace(go.Scatter(x=[1896, 2016], 
                             y=[.5, .5],
                             mode='lines',
                             showlegend=False,
                             marker_color="#222"),
                             row=row,
                             col=col)

    fig.update_xaxes(range=[1896, 2020], row=row, col=col)
    fig.update_yaxes(range=[0, .7], row=row, col=col)


fig.update_layout(height=1200, width=1000)
fig.show()

The nations above were highlighted because of the ammount of medals they won, but this excludes big teams such as India (120 people at Rio 2016) or Turkey (103). What if we select countries by size of delegation in the last games?

In [257]:
# Computes a totals columns
noc_games_sex["total"] = noc_games_sex.F.fillna(0) + noc_games_sex.M.fillna(0)

In [258]:
# Extracts big delegations in Rio -- 50 athletes or more
big_teams = (noc_games_sex[(noc_games_sex.Year==2016) & (noc_games_sex.total >= 50)]
     .sort_values(by='total', ascending=False)
     .NOC
     .unique())

In [259]:
# How many do we have
len(big_teams)

58

In [260]:
# Plots many small multiples
positions = [
             (1,1), (1,2), (1,3), (1,4), (1,5),
             (2,1), (2,2), (2,3), (2,4), (2,5),
             (3,1), (3,2), (3,3), (3,4), (3,5),
             (4,1), (4,2), (4,3), (4,4), (4,5),
             (5,1), (5,2), (5,3), (5,4), (5,5),
             (6,1), (6,2), (6,3), (6,4), (6,5),
             (7,1), (7,2), (7,3), (7,4), (7,5),
             (8,1), (8,2), (8,3), (8,4), (8,5),
             (9,1), (9,2), (9,3), (9,4), (9,5),
            (10,1), (10,2), (10,3), (10,4), (10,5),
            (11,1), (11,2), (11,3), (11,4), (11,5),
            (12,1), (12,2), (12,3),# (12,4), (12,5)
    ]


fig = make_subplots(rows=12, cols=5, subplot_titles=big_teams)


assert len(big_teams) == len(positions)

for NOC, position in zip(big_teams, positions):
    
    subset = noc_games_sex[noc_games_sex.NOC==NOC]
    subset = subset.sort_values(by='Year')
    
    row = position[0]
    col = position[1]
    
    #print(row, col)

    fig.add_trace(go.Scatter(x=subset.Year,
                    y=subset.Percentage_women,
                    mode='lines+markers',
                    marker_color='rgb(55, 83, 109)',
                    showlegend=False),
                    row=row,
                    col=col,
                )
    
    # Adds a 50% line
    fig.add_trace(go.Scatter(x=[1896, 2016], 
                             y=[.5, .5],
                             mode='lines',
                             showlegend=False,
                             marker_color="#222"),
                             row=row,
                             col=col)

    fig.update_xaxes(range=[1896, 2020], row=row, col=col)
    fig.update_yaxes(range=[0, .7], row=row, col=col)


fig.update_layout(height=2400, width=1000)
fig.show()

#### In those nations, were are women overperforming or underperforming?
If men and women in a team are equally competitive, we would assume that the percentage of medals each one wins is proportional to its share of athletes. Is that so?

This can be calculated simply by computing the expected ammount of medals if distribution was proportional to share of team divided by the actual ammount they got.

There is something we need to consider, however: medals in team sports, where there are many athletes, but a single medal is awarded. This can skew data and is a caveat that we need to highlight.

In [264]:
# TO DO:

## Keep valid duplicated medals (e.g team sports, doubles or even draws)

In [22]:
# Removes mixed events
valid_medals = df[~df.Event.str.contains("Mixed")].reset_index(drop=True)

# Keeps a single medal of each kind for event - note that this removes some valid bronze medals
# and will need fixing later
valid_medals = df.drop_duplicates(subset=['Event', 'Medal', 'Year']).reset_index(drop=True)

# Groups the data by country and gender
valid_medals = (valid_medals.groupby(["NOC", "Sex", "Medal", "Year"])
        .ID
        .count()
        .reset_index()
        .rename(columns={"ID":"count"})
        .pivot_table(index=['NOC', 'Year'], columns='Sex', values='count', aggfunc='sum')
        .reset_index()
        .fillna(0))

# Renames columns to avoid confusion
valid_medals = valid_medals.rename(columns={"F":"F_medals", "M":"M_medals"})

# Computes total medals
valid_medals["Total_medals"] = valid_medals.F_medals.fillna(0) + valid_medals.M_medals.fillna(0)

# Crates a field for women percentage
valid_medals["Percentage_women_medals"] = valid_medals["F_medals"] / valid_medals["Total_medals"]

In [23]:
valid_medals[valid_medals.Year==2016].sort_values(by='Total_medals', ascending=False)

Sex,NOC,Year,F_medals,M_medals,Total_medals,Percentage_women_medals
1204,USA,2016,64.0,55.0,119.0,0.537815
221,CHN,2016,39.0,28.0,67.0,0.582090
442,GBR,2016,25.0,40.0,65.0,0.384615
988,RUS,2016,29.0,26.0,55.0,0.527273
407,FRA,2016,11.0,30.0,41.0,0.268293
668,JPN,2016,16.0,23.0,39.0,0.410256
473,GER,2016,18.0,20.0,38.0,0.473684
59,AUS,2016,13.0,16.0,29.0,0.448276
631,ITA,2016,10.0,18.0,28.0,0.357143
205,CAN,2016,14.0,6.0,20.0,0.700000


In [24]:
# Total medals by 
noc_games_sex = noc_games_sex.rename(columns={"F":"F_athletes", "M":"M_athletes", 
                                              "Percentage_women":"Percentage_women_athletes",
                                              "total":"Total_athletes"})

In [25]:
# Merges both
valid_medals = valid_medals.merge(noc_games_sex, on=['NOC', 'Year'])

In [26]:
# Predicted female medals
valid_medals["Predicted_F_medals"] = valid_medals['Total_medals'] * valid_medals['Percentage_women_athletes']

In [27]:
# Excess medals 
valid_medals["Excess_F_medals"] = valid_medals["F_medals"] / valid_medals["Predicted_F_medals"]

In [28]:
valid_medals[valid_medals.NOC=='ROU']

Sex,NOC,Year,F_medals,M_medals,Total_medals,Percentage_women_medals,Games,F_athletes,M_athletes,Percentage_women_athletes,Total_athletes,Predicted_F_medals,Excess_F_medals
944,ROU,1924,0.0,1.0,1.0,0.000000,1924 Summer,NaN,35.0,NaN,35.0,NaN,NaN
945,ROU,1936,0.0,1.0,1.0,0.000000,1936 Summer,2.0,52.0,0.037037,54.0,0.037037,0.000000
946,ROU,1952,0.0,4.0,4.0,0.000000,1952 Summer,11.0,103.0,0.096491,114.0,0.385965,0.000000
947,ROU,1956,3.0,10.0,13.0,0.230769,1956 Summer,11.0,39.0,0.220000,50.0,2.860000,1.048951
948,ROU,1960,4.0,5.0,9.0,0.444444,1960 Summer,16.0,82.0,0.163265,98.0,1.469388,2.722222
949,ROU,1964,5.0,7.0,12.0,0.416667,1964 Summer,30.0,108.0,0.217391,138.0,2.608696,1.916667
950,ROU,1968,6.0,9.0,15.0,0.400000,1968 Summer,16.0,66.0,0.195122,82.0,2.926829,2.050000
951,ROU,1972,4.0,12.0,16.0,0.250000,1972 Summer,27.0,132.0,0.169811,159.0,2.716981,1.472222
952,ROU,1976,8.0,17.0,25.0,0.320000,1976 Summer,54.0,103.0,0.343949,157.0,8.598726,0.930370
953,ROU,1980,10.0,12.0,22.0,0.454545,1980 Summer,74.0,154.0,0.324561,228.0,7.140351,1.400491


In [49]:
# Let's plot stuff
# Plots many small multiples
positions = [
             (1,1), (1,2), (1,3), (1,4), (1,5),
             (2,1), (2,2), (2,3), (2,4), (2,5),
             (3,1), (3,2), (3,3), (3,4), (3,5),
             (4,1), (4,2), (4,3), (4,4), (4,5),
             (5,1), (5,2), (5,3), (5,4), (5,5),
             (6,1), (6,2), (6,3), (6,4), (6,5),
             (7,1), (7,2), (7,3), (7,4), (7,5),
             (8,1), (8,2), (8,3), (8,4), (8,5),
             (9,1), (9,2), (9,3), (9,4), (9,5),
            (10,1), (10,2), (10,3), (10,4), (10,5),
            (11,1), (11,2), (11,3), (11,4), (11,5),
            (12,1), (12,2), (12,3),# (12,4), (12,5)
    ]


fig = make_subplots(rows=12, cols=5, subplot_titles=big_teams)


assert len(big_teams) == len(positions)

for NOC, position in zip(big_teams, positions):
    
    subset = valid_medals[valid_medals.NOC==NOC]
    subset = subset.sort_values(by='Year')
    
    row = position[0]
    col = position[1]
    
    #print(row, col)

    fig.add_trace(go.Scatter(x=subset.Year,
                    y=subset.Excess_F_medals,
                    mode='lines+markers',
                    marker_color='rgb(55, 83, 109)',
                    showlegend=False),
                    row=row,
                    col=col,
                )
    
    # Adds an 'as expected' line
    fig.add_trace(go.Scatter(x=[1896, 2016], 
                             y=[1, 1],
                             mode='lines',
                             showlegend=False,
                             marker_color="#222"),
                             row=row,
                             col=col)

    fig.update_xaxes(range=[1947, 2020], row=row, col=col)
    fig.update_yaxes(range=[-.1, 4], row=row, col=col)


fig.update_layout(height=2400, width=1000)
fig.show()

Now that we finished exploring the dataset, we can save the resulting CSVs and work on the visualizations in a cleaner notebook.